In [1]:
# import
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings('ignore')
#import matplotlib.pyplot as plt
import itertools
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
# Machine Learning libraries.
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA


/Users/raviagrawal/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/raviagrawal/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Helper function to locate full path of file with file_id
files_lookup = {}
for files in glob.glob("./csv/*.csv"):
    files_lookup[files.split("_")[2]] = files
print("example")
files_lookup["819087"]

example


'./csv/File_ID_819087_Index_33564154.csv'

In [3]:
#Files to be loaded
driverDemo =  pd.read_excel("InSight Tables_Keys/DriverDemographic.xlsx")
tripsum =  pd.read_excel("InSight Tables_Keys/TripSummaryMeasures.xlsx")
event =  pd.read_excel("InSight Tables_Keys/eventDetails.xlsx")
eventUpdate = pd.read_excel("InSight Tables_Keys/eventDetailsUpdate.xlsx")
visualCognitive = pd.read_excel("InSight Tables_Keys/VisualAndCognitiveTests.xlsx")
medical = pd.read_excel('InSight Tables_Keys/MedicalConditions&MedicationsData.xlsx')
clockDrawingTests = pd.read_excel("InSight Tables_Keys/clockDrawingTests.xlsx")
sleepTable = pd.read_excel("InSight Tables_Keys/SleepHabitsQuestionnaire.xlsx")
DriverExit = pd.read_excel("InSight Tables_Keys/DriverExitInterview.xlsx")
DrivingHistory = pd.read_excel("InSight Tables_Keys/DrivingHistory.xlsx")
DrivingBehavior = pd.read_excel("InSight Tables_Keys/DriverBehaviorQuestionnaire.xlsx")

In [4]:
# Driving Behaviour
DrivingBehavior.fillna(0, inplace=True)

# pass on the turning vehicle 
DrivingBehavior.passTrnVeh[DrivingBehavior.passTrnVeh == 'Never'] = 0
DrivingBehavior.passTrnVeh[DrivingBehavior.passTrnVeh == 'Occasionally'] = 1
DrivingBehavior.passTrnVeh[DrivingBehavior.passTrnVeh == 'Hardly Ever'] = 2
 
# Road Rage
DrivingBehavior.rage[DrivingBehavior.rage == 'Never'] = 0
DrivingBehavior.rage[DrivingBehavior.rage == 'Occasionally'] = 1
DrivingBehavior.rage[DrivingBehavior.rage == 'Quite Often'] = 2
DrivingBehavior.rage[DrivingBehavior.rage == 'Hardly Ever'] = 3

# Wrong Way
DrivingBehavior.wrngWay[DrivingBehavior.wrngWay == 'Never'] = 0
DrivingBehavior.wrngWay[DrivingBehavior.wrngWay == 'Occasionally'] = 1
DrivingBehavior.wrngWay[DrivingBehavior.wrngWay == 'Hardly Ever'] = 2

# Miss lead vehicle
DrivingBehavior.missLead[DrivingBehavior.missLead == 'Never'] = 0
DrivingBehavior.missLead[DrivingBehavior.missLead == 'Occasionally'] = 1
DrivingBehavior.missLead[DrivingBehavior.missLead == 'Hardly Ever'] = 2

# Wrong lane at the intersection
DrivingBehavior.wrngLnInt[DrivingBehavior.wrngLnInt == 'Never'] = 0
DrivingBehavior.wrngLnInt[DrivingBehavior.wrngLnInt == 'Occasionally'] = 1
DrivingBehavior.wrngLnInt[DrivingBehavior.wrngLnInt == 'Quite Often'] = 2
DrivingBehavior.wrngLnInt[DrivingBehavior.wrngLnInt == 'Hardly Ever'] = 3

In [5]:
# Driving History
DrivingHistory = DrivingHistory[["anonymousParticipantID", 'numViol','numCrash']]
DrivingHistory.fillna(0, inplace=True)

DrivingHistory.numViol[DrivingHistory.numViol == '0'] = 0
DrivingHistory.numViol[DrivingHistory.numViol == '1'] = 1
DrivingHistory.numViol[DrivingHistory.numViol == '2 or More'] = 2

DrivingHistory.numCrash[DrivingHistory.numCrash == '0'] = 0
DrivingHistory.numCrash[DrivingHistory.numCrash == '1'] = 1
DrivingHistory.numCrash[DrivingHistory.numCrash == '2 or More'] = 2

In [6]:
# Driving Exit 
DriverExit.fillna(0, inplace=True)
a = []
for i in DriverExit.driverRestriction:
    if i:
        if "left" in i:
            a.append(2)
        
        else:
            a.append(1)
    else:
        a.append(0)
        
DriverExit.driverRestriction = a        
DriverExit = DriverExit[["anonymousParticipantID", "driverRestriction"]]

In [7]:
print("Near Crash: ", len(event[event.eventSeverity1 == "Near-Crash"]))
print("Crash: ", len(event[event.eventSeverity1 == "Crash"]))

Near Crash:  296
Crash:  81


In [8]:
# Sleep Table processing 
sleepTable = sleepTable.drop(["currWorkStat", "nodOffWhlDrvngFreq", "slpDur", 'awkWllBng', 'awkFnctng', 'awkSlpiness'], axis = 1)
sleepTable.fillna(0 , inplace=True)

sleepTable.slpSched[sleepTable.slpSched == "N"] = 0
sleepTable.slpSched[sleepTable.slpSched != 0] = 1

# Sleep type
sleepTable.slperTyp[sleepTable.slperTyp == "Light"] = 1
sleepTable.slperTyp[sleepTable.slperTyp == "Normal"] = 2
sleepTable.slperTyp[sleepTable.slperTyp == "Heavy"] = 3

# nodOffWhlDrvng
sleepTable.nodOffWhlDrvng[sleepTable.nodOffWhlDrvng == "N"] = 0
sleepTable.nodOffWhlDrvng[sleepTable.nodOffWhlDrvng != 0] = 1

#qltyofSlp
sleepTable.qltyofSlp[sleepTable.qltyofSlp == "Satisfactory"] = 1
sleepTable.qltyofSlp[(sleepTable.qltyofSlp != 1) & (sleepTable.qltyofSlp != 0)] = 2


In [9]:
# Visual Table processing 
visualCognitive = visualCognitive[["anonymousParticipantID", "dhiVisualSearchScore", "dhiUFOVScore"]]
visualCognitive.fillna(3, inplace=True)

visualCognitive.dhiVisualSearchScore[visualCognitive.dhiVisualSearchScore == "none"] = 0
visualCognitive.dhiVisualSearchScore[visualCognitive.dhiVisualSearchScore == "mild"] = 1
visualCognitive.dhiVisualSearchScore[visualCognitive.dhiVisualSearchScore == "serious"] = 2

visualCognitive.dhiUFOVScore[visualCognitive.dhiUFOVScore == "none"] = 0
visualCognitive.dhiUFOVScore[visualCognitive.dhiUFOVScore == "mild"] = 1
visualCognitive.dhiUFOVScore[visualCognitive.dhiUFOVScore == "serious"] = 2

In [10]:
# Processing the medical related data bases
medical = medical[["anonymousParticipantID", "hear", "heart","resp", "brain", "vasc", "nrvsandSlp", "metab",
                   "limFlex", "sevArthr", "psych", "multMedConds",
                    "multMeds"]]
medical.fillna(0, inplace=True)

In [11]:
# hear Either have hear problem or not 
medical.hear[medical.hear != 0] = 1

# Heart Either have heart problem or not 
medical.heart[medical.heart != 0] = 1

# Resp
medical.resp[medical.resp == "Asthma;"] = 1
medical.resp[medical.resp == "Chronic Obstructive Pulmonary Disease (COPD);"] = 2

#brain either have or not 
medical.brain[medical.brain != 0] = 1

#vascular
medical.vasc[medical.vasc != 0] = 1

#nervous system
medical.nrvsandSlp[medical.nrvsandSlp != 0] = 1

# Metab
medical.metab[medical.metab == 'Hypothyroidism;'] = 1
medical.metab[medical.metab == 'Hyperthyroidism;'] = 1
medical.metab[(medical.metab != 0) & (medical.metab != 1)] = 2

# limited flex
medical.limFlex[medical.limFlex == "N"] = 0
medical.limFlex[medical.limFlex != 0] = 1

# sevArthr
medical.sevArthr[medical.sevArthr == "N"] = 0
medical.sevArthr[medical.sevArthr != 0] = 1

# psych
medical.psych[medical.psych != 0] = 1

# multMedConds
medical.multMedConds[medical.multMedConds == "N"] = 0
medical.multMedConds[medical.multMedConds != 0] = 1

#multMeds
medical.multMeds[medical.multMeds == "N"] = 0
medical.multMeds[medical.multMeds != 0] = 1


In [12]:
def DrivingBehaviour_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    passTrnVeh = DrivingBehavior[DrivingBehavior.anonymousParticipantID == id]["passTrnVeh"].values[0]
    rage = DrivingBehavior[DrivingBehavior.anonymousParticipantID == id]["rage"].values[0]
    wrngWay = DrivingBehavior[DrivingBehavior.anonymousParticipantID == id]["wrngWay"].values[0]
    missLead = DrivingBehavior[DrivingBehavior.anonymousParticipantID == id]["missLead"].values[0]
    wrngLnInt = DrivingBehavior[DrivingBehavior.anonymousParticipantID == id]["wrngLnInt"].values[0]

    return passTrnVeh, rage, wrngWay, missLead, wrngLnInt

In [13]:
def DrivingHistory_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    numViol = DrivingHistory[DrivingHistory.anonymousParticipantID == id]["numViol"].values[0]
    numCrash = DrivingHistory[DrivingHistory.anonymousParticipantID == id]["numCrash"].values[0]
    return numViol, numCrash

In [14]:
def Driver_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    driverRestriction = DriverExit[DriverExit.anonymousParticipantID == id]["driverRestriction"].values[0]
    return driverRestriction

In [15]:
def sleep_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    temp = sleepTable[sleepTable.anonymousParticipantID == id]
    slpSched = temp["slpSched"].values[0]
    slperTyp = temp["slperTyp"].values[0]
    nodOffWhlDrvng = temp["nodOffWhlDrvng"].values[0]
    qltyofSlp = temp["qltyofSlp"].values[0]
    return slpSched, slperTyp, nodOffWhlDrvng, qltyofSlp

In [16]:
def getCrashStatus(ids):
    crash_status = event[event.displayTripId==ids].eventSeverity1.values[0]
    return crash_status

In [17]:
def clock_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    temp_data = clockDrawingTests[clockDrawingTests.anonymousParticipantID == id]["clockDrawingScore"].values[0]
    return temp_data

In [18]:
def medical_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    temp_data = medical[medical.anonymousParticipantID == id]
    hear = temp_data["hear"].values[0]
    heart = temp_data["heart"].values[0]
    resp = temp_data["resp"].values[0]
    brain = temp_data["brain"].values[0]
    vasc = temp_data["vasc"].values[0]
    nrvsandSlp = temp_data["nrvsandSlp"].values[0]
    metab = temp_data["metab"].values[0]
    limFlex = temp_data["limFlex"].values[0]
    sevArthr = temp_data["sevArthr"].values[0]
    psych = temp_data["psych"].values[0]
    multMedConds = temp_data["multMedConds"].values[0]
    multMeds = temp_data["multMeds"].values[0]
    return hear, heart, resp, brain, vasc, nrvsandSlp, metab, limFlex, sevArthr, psych, multMedConds, multMeds

In [19]:
def visual_databases(id):
    id = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID'].values[0]
    temp_data = visualCognitive[visualCognitive['anonymousParticipantID'] == id]
    dhiVisual = temp_data["dhiVisualSearchScore"].values[0]
    dhiFOV = temp_data["dhiUFOVScore"].values[0]
    return dhiVisual, dhiFOV 

In [20]:
def get_age(id):
    # Function to print the age of person
    a = tripsum[tripsum["displayTripID"] == id]['anonymousParticipantID']
    a_string = driverDemo[driverDemo["anonymousParticipantID"] == a.values[0]].ageGroup.values[0]
    gender = driverDemo[driverDemo["anonymousParticipantID"] == a.values[0]].sex.values[0]
    gender = 0 if gender == "M" else 1
    a_string = a_string.split("-")
    return int(max(a_string)), gender
print("Age of the person is: ", get_age(14767568))

Age of the person is:  (49, 1)


In [21]:
def getEventUpdate(tripid):
    preIncedent = eventUpdate[eventUpdate.displayTripId == tripid]['preIncidentManeuver'].values[0]
    maneuver  = eventUpdate[eventUpdate.displayTripId == tripid]['maneuverJudgment'].values[0]
    precipiting = eventUpdate[eventUpdate.displayTripId == tripid]['precipitatingEvent'].values[0]
    return preIncedent, maneuver, precipiting

# Examples
print(getEventUpdate(4952738))

('Starting in traffic lane', 'Safe and legal', 'Other vehicle ahead - slowed and stopped 2 seconds or less')


In [22]:
coded_files = pd.read_excel("SHRP2 Data Recording_File_download_from_Qualtric_May4_2018_tz.xlsx", header = 0)

#Droping the unneccessary columns
coded_files = coded_files.drop(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Scorer', 'Scorer - Topics',  'comment'], axis = 1)


In [23]:
print(coded_files.shape[0])
# Removing the initial test values
coded_files = coded_files[8:]

872


In [24]:

# printing the number of crashes
print("Number of crashes: ",coded_files[coded_files.crash == 2].shape)

# Not Selecting incomplete videos
coded_files = coded_files[(coded_files["incomplete_video"] != 2)]
coded_files.shape

Number of crashes:  (14, 20)


(745, 20)

In [25]:
# Few new column added
coded_files["Avg_Acceleration"]  = None
coded_files["std_Acceleration"]  = None
coded_files["Avg_speed"]  = None
coded_files["std_speed"]  = None
coded_files["max_speed"] = None
coded_files["max_acceleration"] = None


coded_files["6_Avg_Acceleration"]  = None
coded_files["6_std_Acceleration"]  = None
coded_files["6_Avg_speed"]  = None
coded_files["6_std_speed"]  = None

coded_files["age"] = None

coded_files["crashStatus"] = None

# addding data from eventDetailsupdate
coded_files['preIncidentManeuver'] = None
coded_files['maneuverJudgment'] = None
coded_files['precipitatingEvent'] = None

# Gender
coded_files["gender"] = None

# Adding medial values
coded_files["hear"]= None
coded_files["heart"]= None
coded_files["resp"]= None
coded_files["brain"]= None
coded_files["vasc"]= None
coded_files["nrvsandSlp"]= None
coded_files["metab"]= None
coded_files["limFlex"]= None
coded_files["sevArthr"]= None
coded_files["psych"]= None
coded_files["multMedConds"]= None
coded_files["multMeds"] = None

# Visual
coded_files["dhiVisual"] = None
coded_files["dhiFOV"] = None

# clockDrawaing
coded_files["clockDrawaing"] = None

# sleep 
coded_files["slpSched"] = None
coded_files["slperTyp"] = None
coded_files["nodOffWhlDrvng"] = None
coded_files["qltyofSlp"] = None


# Driver Exit
coded_files["driverRestriction"] = None

# Driving History
coded_files["numViol"] = None
coded_files["numCrash"] = None

# Drivers behaviour
coded_files["passTrnVeh"] = None
coded_files["rage"] = None
coded_files["wrngWay"] = None
coded_files["missLead"] = None
coded_files["wrngLnInt"] = None


In [26]:
from tqdm import tqdm
not_found = []
for files in tqdm(coded_files.File_id):
    # Reading time 
    time_enter = coded_files[coded_files.File_id==files]["time_of_entering"]
    time_leave = coded_files[coded_files.File_id==files]["time_of_exiting"]

    # Reading files
    try:
        current_vehicle_data = pd.read_csv(files_lookup[str(files)])
    except:
        not_found.append(files)
        continue
    coded_files["crashStatus"][coded_files.File_id==files] = getCrashStatus(files)
    age, gender = get_age(files)
    coded_files["age"][coded_files.File_id==files] = age
    coded_files["gender"][coded_files.File_id==files] = gender
    
    
    preIncedent, maneuver, precipiting = getEventUpdate(int(files))
    
    coded_files['preIncidentManeuver'][coded_files.File_id==files] = preIncedent
    coded_files['maneuverJudgment'][coded_files.File_id==files] = maneuver
    coded_files['precipitatingEvent'][coded_files.File_id==files] = precipiting
    # Will update only if the time is not nan or other faulty file
    
    #clock update
    try:
        clockDraw = clock_databases(int(files))
        coded_files["clockDrawaing"][coded_files.File_id==files] = clockDraw
    except:
        print(files)
    
    # medical
    try:
        hear, heart, resp, brain, vasc, nrvsandSlp, metab, limFlex, sevArthr, psych, multMedConds, multMeds = medical_databases(int(files))
        coded_files["hear"][coded_files.File_id==files]= hear
        coded_files["heart"][coded_files.File_id==files]= heart
        coded_files["resp"][coded_files.File_id==files]= resp
        coded_files["brain"][coded_files.File_id==files]= brain
        coded_files["vasc"][coded_files.File_id==files]= vasc
        coded_files["nrvsandSlp"][coded_files.File_id==files]= nrvsandSlp
        coded_files["metab"][coded_files.File_id==files]= metab
        coded_files["limFlex"][coded_files.File_id==files]= limFlex
        coded_files["sevArthr"][coded_files.File_id==files]= sevArthr
        coded_files["psych"][coded_files.File_id==files]= psych
        coded_files["multMedConds"][coded_files.File_id==files]= multMedConds
        coded_files["multMeds"][coded_files.File_id==files] = multMeds
    except:
        print(files)
    
    #visual
    try:
        dhiVisual, dhiFOV = visual_databases(int(files))
        coded_files["dhiVisual"][coded_files.File_id==files] = dhiVisual
        coded_files["dhiFOV"][coded_files.File_id==files] = dhiFOV
    except:
        print(files, age)
    
    
    # sleep
    try:
        slpSched, slperTyp, nodOffWhlDrvng, qltyofSlp = sleep_databases(int(files))
        coded_files["slpSched"][coded_files.File_id==files] = slpSched
        coded_files["slperTyp"][coded_files.File_id==files] = slperTyp
        coded_files["nodOffWhlDrvng"][coded_files.File_id==files] = nodOffWhlDrvng
        coded_files["qltyofSlp"][coded_files.File_id==files] = qltyofSlp
    except:
        print(files)
    
    try:
        driverRestriction = Driver_databases(int(files))
        coded_files["driverRestriction"][coded_files.File_id==files] = driverRestriction
    except:
        print(files)
        
    try:
        numViol, numCrash = DrivingHistory_databases(int(files))
        coded_files["numViol"][coded_files.File_id==files] = numViol
        coded_files["numCrash"][coded_files.File_id==files] = numCrash
    except:
        print(files)
        
    try:
        passTrnVeh, rage, wrngWay, missLead, wrngLnInt = DrivingBehaviour_databases(int(files))
        coded_files["passTrnVeh"][coded_files.File_id==files] = passTrnVeh
        coded_files["rage"][coded_files.File_id==files] = rage
        coded_files["wrngWay"][coded_files.File_id==files] = wrngWay
        coded_files["missLead"][coded_files.File_id==files] = missLead
        coded_files["wrngLnInt"][coded_files.File_id==files] = wrngLnInt
        
    except:
        print(files)

    if not (time_enter.isnull().values[0]) and (files != 16990196) and (files != 16990196):
        # Converting the time to int
        
        time_enter = int(time_enter.values[0])
        time_leave = int(time_leave.values[0])
        
        # Average and std in the intersection
        average_curr_vech_data = current_vehicle_data[(current_vehicle_data['vtti.timestamp'] >= time_enter) & (current_vehicle_data['vtti.timestamp'] <= time_leave)].mean()
        std_curr_vech_data = current_vehicle_data[(current_vehicle_data['vtti.timestamp'] >= time_enter) & (current_vehicle_data['vtti.timestamp'] <= time_leave)].std()
        
        # Max of speed and acceleration
        max_curr_vech_data = current_vehicle_data[(current_vehicle_data['vtti.timestamp'] >= time_enter) & (current_vehicle_data['vtti.timestamp'] <= time_leave)].max()
        
        coded_files["Avg_Acceleration"][coded_files.File_id==files] = abs(average_curr_vech_data["vtti.accel_x"])
        coded_files["std_Acceleration"][coded_files.File_id==files] = abs(std_curr_vech_data["vtti.accel_x"])
        coded_files["Avg_speed"][coded_files.File_id==files] = average_curr_vech_data["vtti.speed_network"]
        coded_files["std_speed"][coded_files.File_id==files] = std_curr_vech_data["vtti.speed_network"]

        coded_files["max_speed"][coded_files.File_id==files] =  max_curr_vech_data["vtti.speed_network"]
        coded_files["max_acceleration"][coded_files.File_id==files] =  max_curr_vech_data["vtti.accel_x"]
        
        # Average and std 6 second before the intersection and all the way to the end
        larger_average_curr_vech_data = current_vehicle_data[(current_vehicle_data['vtti.timestamp'] >=(time_enter - 6000)) & (current_vehicle_data['vtti.timestamp'] <= time_leave)].mean()
        larger_std_curr_vech_data = current_vehicle_data[(current_vehicle_data['vtti.timestamp'] >=(time_enter - 6000)) & (current_vehicle_data['vtti.timestamp'] <= time_leave)].std()
        
        coded_files["6_Avg_Acceleration"][coded_files.File_id==files] = abs(larger_average_curr_vech_data["vtti.accel_x"])
        coded_files["6_std_Acceleration"][coded_files.File_id==files] = abs(larger_std_curr_vech_data["vtti.accel_x"])
        coded_files["6_Avg_speed"][coded_files.File_id==files] = larger_average_curr_vech_data["vtti.speed_network"]
        coded_files["6_std_speed"][coded_files.File_id==files] = larger_std_curr_vech_data["vtti.speed_network"]
        
        

  0%|          | 2/745 [00:00<01:47,  6.92it/s]

948646


  1%|          | 4/745 [00:00<01:52,  6.57it/s]

1091854
1091854


  1%|          | 8/745 [00:01<01:48,  6.77it/s]

...

 96%|█████████▌| 715/745 [01:49<00:04,  6.54it/s]

98235356
98235356


 97%|█████████▋| 723/745 [01:51<00:03,  6.51it/s]

100489346


100%|██████████| 745/745 [01:54<00:00,  6.48it/s]


In [27]:
not_found

[nan, nan]

In [28]:
leftTurn = coded_files[coded_files.vehicle_movement==2]

In [29]:
leftTurn.to_excel("leftturn_new.xlsx")

In [ ]:
leftTurn.shape

In [ ]:
leftTurn.to_excel("leftturn.xlsx")

In [ ]:
leftTurn = pd.read_excel("leftturn.xlsx")

In [ ]:
leftTurn.head(5)

In [ ]:
data = leftTurn.drop(["File_id","time_of_entering","time_of_exiting","precipitatingEvent","preIncidentManeuver", "crash", "maneuverJudgment", "incomplete_video", "vehicle_movement", 'intersection_type'], axis = 1)

In [ ]:
data.columns

#  

In [ ]:
coded_files.dropna(inplace=True)

In [ ]:
coded_files.columns

In [ ]:
coded_files = coded_files[[ 'age',
       'crashStatus' ,'hear', 'heart', 'resp', 'brain', 'vasc',
       'nrvsandSlp', 'metab', 'limFlex', 'sevArthr', 'psych', 'multMedConds',
       'multMeds']]

# Helper Function to fit to the classifier, print training and validation accuracy, precision, recall, f1 Score, Confusion Matrix.

#  

In [ ]:
y = data["crashStatus"]
data = data.drop( 'crashStatus', axis=1)
y[y=="Balanced-Sample Baseline"] = False
y[y != False] = True
y  = np.asarray(y,dtype=np.float64)
filledData = data.where(pd.notna(data), data.mean(), axis='columns')
filledData = (filledData - filledData.mean())/filledData.std()
print("Number of crashs events: ", sum(y))
print("Class Balance with Crash and no Crash: ", sum(y)/len(y))
# from sklearn.feature_selection import SelectKBest,f_classif
# selector = SelectKBest(f_classif, k=10).fit(filledData,y)
# featurefilledData = selector.transform(filledData)
X_train, X_test, y_train, y_test = train_test_split(filledData, y, train_size=0.7)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    


def Classifier_performance(clf, target_names, y, filledData=filledData, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, title="Prediction confusion matrix"):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    Y_train_pred = clf.predict(X_train)
    print("Training Accuracy Score: ", accuracy_score(Y_train_pred , y_train))
    print("validation Accuracy Score:", accuracy_score(y_pred, y_test))
    print("Classification Report: ")
    print(classification_report(y_test, y_pred, target_names=target_names))
    cm = confusion_matrix(y_test, y_pred)
    # Plot normalized confusion matrix
    #plt.figure()
    plot_confusion_matrix(cm, classes=target_names, normalize=False,
                          title=title)

    plt.show()
    plt.close()
    
    pca = PCA(n_components=2)
    filledData_lower = pca.fit_transform(filledData)
    y = pd.Series(y)
    filledData_lower = pd.DataFrame(filledData_lower)
    filledData_lower['labels'] = y.values
    X = filledData_lower
    y = filledData_lower["labels"]
    y  = np.asarray(y,dtype=np.float64)
    crash = filledData_lower[filledData_lower["labels"] == 1]
    no_crash = filledData_lower[filledData_lower["labels"] == 0]
    plt.plot(crash[0], crash[1], "*", label=target_names[0])
    plt.plot(no_crash[0], no_crash[1], "o", label=target_names[1])
    plt.legend()
    plt.show()
    plt.close()

# Predicting Crash risk of all the drivers using oridinary least square penality considering crash and near crash as one entry.

In [ ]:
clf = LogisticRegression(max_iter=1000, penalty='l1')

In [ ]:
Classifier_performance(clf, ["No_crash", "Crash"], y, filledData=filledData, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test )

In [ ]:
import statsmodels.api as sm
from scipy import stats
est = sm.OLS(y, filledData.astype(float))
est2 = est.fit()
print(est2.summary())

#  

# Predicting Crash risk for including just crash and baseline condition. (Near crash not considered) using the vehicle profile from both young and old driver.


In [ ]:
leftTurn = pd.read_excel("leftturn.xlsx")

# removing the near crash files.
leftTurn = leftTurn[leftTurn["crashStatus"] != 'Near-Crash'] 

data = leftTurn.drop(["File_id","time_of_entering","time_of_exiting","precipitatingEvent",
                      "preIncidentManeuver", "crash", "maneuverJudgment", "incomplete_video", 
                      "vehicle_movement", 'intersection_type'], axis = 1)

# baselines = data[data["crashStatus"]=="Balanced-Sample Baseline"][:60]
# crash = data[data["crashStatus"]=="Crash"]
# data = pd.concat([baselines, crash])

y = data["crashStatus"]
data = data.drop( 'crashStatus', axis=1)
y[y=="Balanced-Sample Baseline"] = False
y[y != False] = True
y  = np.asarray(y,dtype=np.float64)
filledData = data.where(pd.notna(data), data.mean(), axis='columns')
filledData = (filledData - filledData.mean())/filledData.std()
print("Number of crashs events: ", sum(y))
print("Class Balance with Crash and Baseline Condition: ", sum(y)/len(y))
# from sklearn.feature_selection import SelectKBest,f_classif
# selector = SelectKBest(f_classif, k=10).fit(filledData,y)
# featurefilledData = selector.transform(filledData)
X_train, X_test, y_train, y_test = train_test_split(filledData, y, train_size=0.7)

In [ ]:
clf = LogisticRegression(max_iter=100, penalty='l1')

In [ ]:
Classifier_performance(clf, ["No_crash", "Crash"], y, filledData=filledData, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test )

#  

In [ ]:
import statsmodels.api as sm
from scipy import stats
est = sm.OLS(y, filledData.astype(float))
est2 = est.fit()
print(est2.summary())

#  

# Predicting if the driver is young or old from their driving data all 275 data used

In [ ]:
# leftTurn = pd.read_excel("leftturn.xlsx")
data = leftTurn.drop(["File_id","time_of_entering","time_of_exiting","precipitatingEvent",
                      "preIncidentManeuver", "crash", "maneuverJudgment", "incomplete_video", 
                      "vehicle_movement", 'intersection_type', "crashStatus"], axis = 1)

filledData = data.where(pd.notna(data), data.mean(), axis='columns')
filledData = (filledData - filledData.mean())/filledData.std()
y = leftTurn["age"]
y[y >= 65] = True
y[y != True] = False
y  = np.asarray(y,dtype=np.float64)

In [ ]:
filledData = filledData.drop(["age"], axis =1)

In [ ]:
print("Number of old drivers: ", sum(y))
print("Class Balance with old drivers and young driver: ", sum(y)/len(y))

In [ ]:
# from sklearn.feature_selection import SelectKBest,f_classif
# selector = SelectKBest(f_classif, k="all").fit(filledData,y)
# featurefilledData = selector.transform(filledData)
#featurefilledData.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(filledData, y, train_size=0.7)

In [ ]:
clf = LogisticRegression(max_iter=1000, penalty='l1')

In [ ]:
Classifier_performance(clf, ["Young", "Old"], y, filledData=filledData, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test )

In [ ]:
import statsmodels.api as sm
from scipy import stats
est = sm.OLS(y, filledData.astype(float))
est2 = est.fit()
print(est2.summary())

#  

#  Model to predict old driver at risk of crash or not

In [ ]:
leftTurn = pd.read_excel("leftturn.xlsx")

old_crash = leftTurn[(leftTurn["age"]>=65) & ((leftTurn["crashStatus"]=="Crash"))]
old_baseline = leftTurn[(leftTurn["age"]>=65) & (leftTurn["crashStatus"]=="Balanced-Sample Baseline")][:30]
leftTurn = pd.concat([old_crash, old_baseline])
data = leftTurn.drop(["File_id","time_of_entering","time_of_exiting","precipitatingEvent",
                      "preIncidentManeuver", "crash", "maneuverJudgment", "incomplete_video", 
                      "vehicle_movement", 'intersection_type'], axis = 1)

y = data["crashStatus"]
data = data.drop( 'crashStatus', axis=1)
y[y=="Balanced-Sample Baseline"] = False
y[y != False] = True
y  = np.asarray(y,dtype=np.float64)
filledData = data.where(pd.notna(data), data.mean(), axis='columns')
filledData = (filledData - filledData.mean())/(filledData.std()+1e-4)
print("Number of crashs events: ", sum(y))
print("Class Balance with Crash and Baseline Condition: ", sum(y)/len(y))
# from sklearn.feature_selection import SelectKBest,f_classif
# selector = SelectKBest(f_classif, k=10).fit(filledData,y)
# featurefilledData = selector.transform(filledData)
X_train, X_test, y_train, y_test = train_test_split(filledData, y, train_size=0.7)

In [ ]:
clf = LogisticRegression(max_iter=100, penalty='l1')
Classifier_performance(clf, ["Old driver not at risk", "Old drivers at risk"], y, filledData=filledData, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test )

In [ ]:
est = sm.OLS(y, filledData.astype(float))
est2 = est.fit()
print(est2.summary())

#  

#  Young drivers at the crash risk or not 

In [ ]:
leftTurn = pd.read_excel("leftturn.xlsx")

young_crash = leftTurn[(leftTurn["age"]<=65) & ((leftTurn["crashStatus"]=="Crash"))]
young_baseline = leftTurn[(leftTurn["age"]<=65) & (leftTurn["crashStatus"]=="Balanced-Sample Baseline")][:20]
leftTurn = pd.concat([young_crash, young_baseline])
data = leftTurn.drop(["File_id","time_of_entering","time_of_exiting","precipitatingEvent",
                      "preIncidentManeuver", "crash", "maneuverJudgment", "incomplete_video", 
                      "vehicle_movement", 'intersection_type'], axis = 1)

y = data["crashStatus"]
data = data.drop( 'crashStatus', axis=1)
y[y=="Balanced-Sample Baseline"] = False
y[y != False] = True
y  = np.asarray(y,dtype=np.float64)
filledData = data.where(pd.notna(data), data.mean(), axis='columns')
filledData = (filledData - filledData.mean())/(filledData.std()+1e-8)
print("Number of crashs events: ", sum(y))
print("Class Balance with Crash and Baseline Condition: ", sum(y)/len(y))
# from sklearn.feature_selection import SelectKBest,f_classif
# selector = SelectKBest(f_classif, k=10).fit(filledData,y)
# featurefilledData = selector.transform(filledData)
X_train, X_test, y_train, y_test = train_test_split(filledData, y, train_size=0.7)

In [ ]:
clf = LogisticRegression(max_iter=100, penalty='l1')
Classifier_performance(clf, ["Old driver not at risk", "Old drivers at risk"], y, filledData=filledData, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test )

In [ ]:
est = sm.OLS(y, filledData.astype(float))
est2 = est.fit()
print(est2.summary())

#  